In [53]:
import torch
from torchvision import models
from torch import optim
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
%matplotlib inline


In [54]:
vgg = models.vgg16(pretrained=True)
#vgg
!ls ../datasets/cat_vs_dog

test  train


In [55]:
root_dir = '../datasets/cat_vs_dog'
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])
train_data = datasets.ImageFolder(root_dir + '/train', transform=simple_transform)
test_data = datasets.ImageFolder(root_dir + '/test', transform=simple_transform)

In [56]:
train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True)

In [57]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True

In [58]:
vgg.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [59]:
for param in vgg.features.parameters(): 
    param.requires_grad = False

In [60]:
vgg.classifier[6].out_features = 2

In [61]:
optimizer = optim.SGD(vgg.classifier.parameters(),lr=0.0001,momentum=0.5)

In [62]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data , target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [63]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]
for epoch in range(1,3):
    epoch_loss, epoch_accuracy = fit(epoch,vgg,train_data_loader,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,vgg,valid_data_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)


training loss is   nan and training accuracy is 9308/22500     41.37


NameError: name 'model' is not defined